# Affine Cipher

Affine cipher is a cipher technique to encrypt plain text using a 2 part key.
Assume we have the key pair (k1, k2) we can encrypt a plain text say P, we calculate the cipher text C as -

C = (P x k1 + k2) mod 26 (assuming we are limited to 26 character alphabet)

## Implementation

Firstly we define an encoding function that will be used to take plaintext and convert it to a 26 character encoding. (By converting all letters to upper case and discarding all remaining characters)

In [1]:
def encode(string):
    result = ''
    for letter in string:
        if letter.isalpha():
            result += letter.upper()
    return result

Lets declare a plain text that we would need to encrypt.

In [2]:
P = 'This is a plain text.'

Encoding this string, we get -

In [3]:
T = encode(P)
print(T)

THISISAPLAINTEXT


Since we will be only having 26 characters, we declare Zp as closed ring of 26 integers.

In [4]:
Zp = Integers(26)
print(Zp)

Ring of integers modulo 26


We now declare a key pair (k1, k2) that we would be using to encrypt the plain text.

In [5]:
from random import choice
k1_domain = [int(x) for x in range(26) if gcd(x, 26) == 1]
k2_domain = [int(x) for x in range(26)]
key = k1, k2 = (Zp(choice(k1_domain)), Zp(choice(k2_domain)))
print('Key -', key)

Key - (17, 2)


Using the definition of affine cypher, we can encrypt each character of text using -

C = (P x k1 + k2) mod 26

In [6]:
def affinecipher(text, cipher_key):
    cipher = ''
    k1, k2 = cipher_key
    for letter in text:
        text_value = Zp(ord(letter) - ord('A')) # convert letter to its numerical value
        
        cipher_value = Zp(text_value * k1 + k2) # convert to cipher value
        
        cipher += chr(int(cipher_value) + ord('A')) # conver cipher value to character
        
    return cipher

def affinedecipher(cipher_text, cipher_key):
    text = ''
    k1, k2 = cipher_key
    for letter in cipher_text:
        cipher_value = Zp(ord(letter) - ord('A')) # convert letter to its numerical value
        
        text_value = Zp((cipher_value - k2) * Zp(k1 ^ -1)) # convert to cipher value
        
        text += chr(int(text_value) + ord('A')) # conver cipher value to character
        
    return text

In [7]:
T = encode(P)
C = affinecipher(T, key)
D = affinedecipher(C, key)
print(f'Given text - "{P}"')
print(f'Encoded - {T}')
print(f'Key - {key}')
print(f'Cipher text - {C}')
print(f'Decipher text - {D}')

Given text - "This is a plain text."
Encoded - THISISAPLAINTEXT
Key - (17, 2)
Cipher text - NRIWIWCXHCIPNSDN
Decipher text - THISISAPLAINTEXT


## Test Against Builtin Cipher

Now, we can test the result against the built in Affine Cipher in sagemath.

In [8]:
A = AffineCryptosystem(AlphabeticStrings())
E = A.encoding(P)
print(f'Text - {P}')
print(f'Encoded - {E}')
print(f'Key - ({k1}, {k2})')
C_test = A.enciphering(int(k1), int(k2), E)
D_test = A.deciphering(int(k1), int(k2), C_test)

# convert to python string
#C_test = str(C_test)
D_test = str(D_test)
print(f'Cipher text - {C_test}')
print(f'Decipher text - {D_test}')

Text - This is a plain text.
Encoded - THISISAPLAINTEXT
Key - (17, 2)
Cipher text - NRIWIWCXHCIPNSDN
Decipher text - THISISAPLAINTEXT


Comparing the built in cipher result with our implementation -

In [9]:
print('Results \t Implementation \t Built-in\n')
print(f'Cipher Text \t {C} \t {C_test}')
print(f'Decipher Text \t {D} \t {D_test}\n')
if C_test == C and D_test == D:
    print('Implementation is CORRECT')
else:
    print('Implementatiokn is INCORRECT')

Results 	 Implementation 	 Built-in

Cipher Text 	 NRIWIWCXHCIPNSDN 	 NRIWIWCXHCIPNSDN
Decipher Text 	 THISISAPLAINTEXT 	 THISISAPLAINTEXT

Implementatiokn is INCORRECT


## Cryptoanalysis

Since we know the key domain for the given cipher algorithm, we can run a brute force attack on the cryptosystem using a list of keys possible within the domain.

In [10]:
key_list = []
for i in k1_domain:
    for j in k2_domain:
        t = (i, j)
        key_list.append((i, j))
print(f'Total keys - {len(key_list)}')
print(key_list)

Total keys - 312
[(1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (3, 16), (3, 17), (3, 18), (3, 19), (3, 20), (3, 21), (3, 22), (3, 23), (3, 24), (3, 25), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (5, 12), (5, 13), (5, 14), (5, 15), (5, 16), (5, 17), (5, 18), (5, 19), (5, 20), (5, 21), (5, 22), (5, 23), (5, 24), (5, 25), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (7, 9), (7, 10), (7, 11), (7, 12), (7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (7, 22), (7, 23), (7, 24), (7, 25), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9), (9, 10

We must now get a list of english words that can be used to detect existence of english words in our bruteforced decipher text.

A good list of english words is here -
https://github.com/dwyl/english-words/blob/master/words_alpha.txt

We download the list of words and convert it to a list

In [27]:
import requests
url = 'https://github.com/aneeshsharma/EnglishWords/raw/main/common3000.txt'

words_file = requests.get(url, allow_redirects=True)
words_file_obj = open('words.txt', 'wb')
words_file_obj.write(words_file.content)
words_file_obj.close()

In [28]:
words = open('words.txt').read().split()

In [29]:
words = [word.upper() for word in words]
print(f'Number of words in dictionary - {len(words)}')

Number of words in dictionary - 3000


In [14]:
def find_words(string):
    l = len(string)
    found = []
    for i in range(l):
        for j in range(i, l):
            word = string[i:j+1]
            if len(word) <= 1:
                continue
            if word in words:
                found.append(string[i:j+1])
    return found

Now, we must try to decipher the encrypted text using the list of keys we have and try to compare and count any english words found in the text. More the words detected, more likely is it that the key is correct.

In [26]:
import re
results = []

for candidate in key_list:
    try:
        candidate_text = affinedecipher(C, candidate)
    except Exception as e:
        print(candidate)
        if (3, 0) == candidate:
            candidate_text = affinedecipher(C, (3, 0))
        else:
            continue
    found = find_words(candidate_text)
    if len(found) > 1:
        print('Possible - ', candidate_text, found)
        results.append(candidate_text)
    

Possible -  NRIWIWCXHCIPNSDN ['NR', 'IW', 'WI', 'IW', 'WC', 'NS', 'SD', 'DN']
Possible -  MQHVHVBWGBHOMRCM ['QH', 'HV', 'HV', 'VB', 'WG', 'HO', 'HOM', 'OM', 'MR', 'RC', 'CM']
Possible -  LPGUGUAVFAGNLQBL ['LP', 'PG', 'GU', 'GUGU', 'UG', 'GU', 'AV', 'FA', 'FAG', 'AG', 'GN', 'NL', 'BL']
Possible -  KOFTFTZUEZFMKPAK ['KO', 'KOFT', 'OF', 'OFT', 'FT', 'FT', 'FM', 'MK', 'PA', 'AK']
Possible -  JNESESYTDYELJOZJ ['NE', 'NESE', 'ES', 'ESE', 'ESES', 'SE', 'ES', 'YT', 'TD', 'DY', 'DYE', 'YE', 'EL', 'JO', 'OZ']
Possible -  IMDRDRXSCXDKINYI ['IM', 'MD', 'DR', 'RD', 'DR', 'XS', 'SC', 'XD', 'DK', 'KI', 'KIN', 'IN', 'NY', 'YI']
Possible -  HLCQCQWRBWCJHMXH ['HL', 'LC', 'CQ', 'CQ', 'WR', 'WC', 'HM']
Possible -  GKBPBPVQAVBIGLWG ['KB', 'BP', 'BP', 'AV', 'VB', 'BI', 'BIG', 'GL', 'WG']
Possible -  FJAOAOUPZUAHFKVF ['JA', 'AO', 'AO', 'UP', 'AH', 'HF', 'KV']
Possible -  EIZNZNTOYTZGEJUE ['ZN', 'ZN', 'NT', 'TO', 'TOY', 'OY', 'YT', 'GE', 'JU']
Possible -  DHYMYMSNXSYFDITD ['HY', 'YM', 'MY', 'MYM', 'YM', 'MS',